<a href="https://colab.research.google.com/github/codelspak/Codelion_weekly_projects/blob/main/WK28_W9_Mini_project__Kaggle_DL_Regression_%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9_%EC%8B%9D%EC%88%98_%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문제상황
평가산식 : MAE(Mean Absolute Error)

# 1. 라이브러리 및 데이터 불러오기


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Codelion_weekly assignments/WK28_WK9_DL_구내식당 식수 인원 예측/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Codelion_weekly assignments/WK28_WK9_DL_구내식당 식수 인원 예측/test.csv')
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Codelion_weekly assignments/WK28_WK9_DL_구내식당 식수 인원 예측/sample_submission.csv')

In [5]:
train.shape, test.shape, submit.shape

((1205, 12), (50, 10), (50, 3))

In [6]:
set(train.columns.tolist()) - set(test.columns.tolist()), submit.columns

({'석식계', '중식계'}, Index(['일자', '중식계', '석식계'], dtype='object'))

# 2. 데이터 EDA


In [8]:
train.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0


In [9]:
train.sample(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
557,2018-05-09,수,2721,86,269,0,0.0,모닝롤/치즈프레즐 우유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 열무된장국 ...,쌀밥/잡곡밥 (쌀:국내산) 돈육김치찌개 탕수어 우엉잡채 도라지무침 열무김치 ...,쌀밥/잡곡밥 (쌀:국내산) 대구매운탕 언양식불고기 견과류조림 청경채겉절이 포...,919.0,413.0
563,2018-05-17,목,2721,71,278,343,0.0,모닝롤/햄치즈샌드 우유/주스 스크램블에그 양송이죽/쌀밥 (쌀:국내산) 선지해장국 ...,쌀밥/잡곡밥 (쌀:국내산) 콩나물국 닭볶음탕 탕평채 영양부추무침 포기김치 (...,멸치주먹밥 (쌀:국내산) 김치국수 깐풍육 꽃맛살샐러드 오복지/바나나 포기김치...,811.0,473.0
381,2017-08-17,목,2645,357,221,380,0.0,모닝롤/치킨샌드 우유/주스 스크램블에그 누룽지탕/쌀밥 (쌀:국내산) 황태국 느타...,쌀밥/잡곡밥 (쌀:국내산) 들깨시래기국 매운닭찜 (닭고기:국내산) 임연수구이 치...,김치볶음밥 (쌀:국내산) 어묵우동 미니햄버거 단호박범벅 요플레 배추겉절이 (...,823.0,507.0


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [27]:
# 컬럼 별 결측치 수 / 비율 확인 함수
def missing_values(dataframe) : 
    no_missing_values = dataframe.isnull().sum()
    prop_missing_values = round(dataframe.isnull().sum()/len(dataframe)*100,1)

    df_null = pd.concat([no_missing_values, prop_missing_values], axis = 1)
    df_null = df_null.rename(columns = {0:'결측치 수', 1:'결측치 비율'}, )

    return df_null

In [28]:
# 결측치 확인
missing_values(train)

,결측치 수,결측치 비율
일자,0,0.0
요일,0,0.0
본사정원수,0,0.0
본사휴가자수,0,0.0
본사출장자수,0,0.0
본사시간외근무명령서승인건수,0,0.0
현본사소속재택근무자수,0,0.0
조식메뉴,0,0.0
중식메뉴,0,0.0
석식메뉴,0,0.0


In [ ]:
missing_values(test)

In [31]:
# 기술통계 
train.describe().round(2)

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계
count,1205.00,1205.00,1205.00,1205.00,1205.00,1205.00,1205.00
mean,2807.82,157.91,241.14,274.12,43.51,890.33,461.77
std,171.26,144.19,43.53,246.24,109.94,209.51,139.18
min,2601.00,23.00,41.00,0.00,0.00,296.00,0.00
25%,2645.00,71.00,217.00,4.00,0.00,758.00,406.00
50%,2760.00,105.00,245.00,299.00,0.00,879.00,483.00
75%,2962.00,185.00,272.00,452.00,0.00,1032.00,545.00
max,3305.00,1224.00,378.00,1044.00,533.00,1459.00,905.00


In [35]:
train.describe(include = ['O'])


,일자,요일,조식메뉴,중식메뉴,석식메뉴
count,1205,1205,1205,1205,1205
unique,1205,5,1204,1198,1168
top,2016-02-01,목,모닝롤/프렌치토스트 우유/주스 계란후라이 누룽지탕/쌀밥 (쌀:국내산) 무채국 김...,쌀밥/잡곡밥 (쌀:국내산) 시금치된장국 훈제오리구이 실곤약무침 무쌈/양파절임 ...,*
freq,1,244,2,2,19


뜬금없지만, 조식메뉴는 있는데 조식계 숫자는 왜 안 줌?

날짜 관련 피쳐들

In [ ]:
# 일자 
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (20,3))
# sns.histplot(data = train[['일자', '중식계']], x = '일자', y = '중식계', ax = ax1)
# sns.histplot(data = train[['일자', '석식계']], x = '일자', y = '석식계',ax = ax2)
# plt.show()

In [36]:
# 날짜 관련 파생변수 생성
train['연도'] = train.일자.map(lambda x : x.split("-")[0])
train['월'] = train.일자.map(lambda x : x.split("-")[1])
train['일'] = train.일자.map(lambda x : x.split("-")[2])
train.iloc[:3, -3:]

,연도,월,일
0,2016,02,01
1,2016,02,02
2,2016,02,03


In [46]:
pd.crosstab(index = train.연도, columns = train.월, margins = True)

월,01,02,03,04,05,06,07,08,09,10,11,12,All
연도,,,,,,,,,,,,,
2016,0,18,22,20,20,21,21,22,19,20,22,22,227
2017,20,20,22,20,19,21,21,22,21,16,22,20,244
2018,22,18,21,21,20,19,22,22,15,17,22,20,239
2019,22,17,20,22,21,19,23,21,19,20,21,21,246
2020,20,20,22,20,19,10,23,20,20,19,21,18,232
2021,17,0,0,0,0,0,0,0,0,0,0,0,17
All,101,93,107,103,99,90,110,107,94,92,108,101,1205


위 테이블의 값은 '샘플 수' 자체를 의미함<br>
1월 / 2월 / 5월 / 6월 / 9월 / 10월 / 12월엔 샘플 수가 약 10% 정도 적은 걸 알 수 있음. 16-20년 5개년 치 임을 고려했을 때, 매년 각 달에 약 2일씩 회사가 쉬었음을 알 수 있음. 그 이유는 다음과 같음.<br>
1. 1월 / 9월 / 10월 : 설날, 추석
2. 2월 : 그냥 다른 달에 비해 일수 자체가 적음
3. 5월 : 근로자의 날, 어린이 날, 부처님 오신 날 등 공휴일이 많음
4. 6월 : **2020년 데이터가 이상**함. 누락된 것 같음. 
    - 근데 결측치 없었음. 따라서 값들에 0 등 의미없는 값이 있는지 확인이 필요할 것 같음
5. 12월 : 2020년 데이터만 추가로 확인해보자


In [52]:
# 2018년 9월 데이터 : 그냥 14일부터 일주일 치 데이터가 누락되었음
train.loc[
          (train.연도 == '2018') & (train.월 == '09'), ['일자', '요일']
].value_counts()

일자          요일
2018-09-03  월     1
2018-09-04  화     1
2018-09-05  수     1
2018-09-06  목     1
2018-09-07  금     1
2018-09-10  월     1
2018-09-11  화     1
2018-09-12  수     1
2018-09-13  목     1
2018-09-14  금     1
2018-09-17  월     1
2018-09-18  화     1
2018-09-19  수     1
2018-09-20  목     1
2018-09-21  금     1
dtype: int64

In [48]:
# 2020년 6월 데이터 : 그냥 13일부터 2주치 데이터가 누락되었음
train.loc[
          (train.연도 == '2020') & (train.월 == '06'), ['일자', '요일']
].value_counts()

일자          요일
2020-06-01  월     1
2020-06-02  화     1
2020-06-03  수     1
2020-06-04  목     1
2020-06-05  금     1
2020-06-08  월     1
2020-06-09  화     1
2020-06-10  수     1
2020-06-11  목     1
2020-06-12  금     1
dtype: int64

In [51]:
# 2020년 12월 데이터 : 28일부터 일주일치 데이터 누락되었음
train.loc[
          (train.연도 == '2020') & (train.월 == '12'), ['일자', '요일']
].value_counts().sort_index()

일자          요일
2020-12-01  화     1
2020-12-02  수     1
2020-12-03  목     1
2020-12-04  금     1
2020-12-07  월     1
2020-12-08  화     1
2020-12-09  수     1
2020-12-10  목     1
2020-12-11  금     1
2020-12-14  월     1
2020-12-15  화     1
2020-12-16  수     1
2020-12-17  목     1
2020-12-18  금     1
2020-12-21  월     1
2020-12-22  화     1
2020-12-23  수     1
2020-12-24  목     1
dtype: int64

In [ ]:
# 요일

### 본사 근무 인원 관련 피쳐들

In [ ]:
# 본사 정원수


In [ ]:
# 본사휴가자수 (-)

In [ ]:
# 본사출장자수 : (-) - 경험 상, 누가 출장오면 오히려 다 같이 외식하러 감

In [ ]:
# 본사시간외근무명령서승인건수 (+)

In [ ]:
# 현본사소속재택근무자수 (-)

### 메뉴 관련 피쳐들

In [ ]:
# 조식메뉴

In [ ]:
# 중식메뉴

In [ ]:
# 석식메뉴

### 식사 인원 데이터

In [ ]:
# 중식계

In [ ]:
# 석식계

In [ ]:
# 

In [ ]:
# 

In [ ]:
# 

In [ ]:
# 